In [1]:
import os
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
os.chdir("..")
import visualizer_helpers as vh

In [2]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [3]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [4]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

In [5]:
base_trip = pd.read_csv(f"{base_dir}/final_trips.csv", 
                        index_col='trip_id', 
                        usecols=['trip_id', 'trip_mode', 'trip_count','trip_num', 'trip_count', 'origin','destination', 'purpose'])
base_trip_idx = base_trip.index
build_trip = pd.read_csv(f"{build_dir}/final_trips.csv", 
                         index_col='trip_id', 
                         usecols=['trip_id', 'trip_mode', 'trip_count','trip_num', 'trip_count', 'origin','destination', 'purpose'])


In [6]:
print(f"""Filtering trips by origin {
    'and' if how_method == 'all' else 'or' if how_method == 'any' else '???'
    } destination TAZ.\nOriginal trips in base: {len(base_trip)}\tbuild: {len(build_trip)}""")

base_trip = base_trip[multi_filter_mazs([base_trip.origin, base_trip.destination])] # base tour in the filtered set

build_trip = build_trip[

    (build_trip.index.isin(base_trip_idx) # originally existed in base
    & build_trip.index.isin(base_trip.index) )    # and was in the current set for the base
 
    |     # OR
    ((~build_trip.index.isin(base_trip_idx)) # is a new tour 
     & multi_filter_mazs([build_trip.origin, build_trip.destination])) # and it's in this set
     
    ]

print(f"After filtering, trips in base: {len(build_trip)}\tbuild: {len(build_trip)}")

Filtering trips by origin or destination TAZ.
Original trips in base: 49	build: 49
After filtering, trips in base: 49	build: 49


## Trip Purpose

In [7]:
print("Lorem ipsum") # Placeholder for CS
# count of intermediate stops with same\different purpose

Lorem ipsum


In [8]:
# To analyze "intermediate" trip purposes, count only:
# Trips on tours multiple stops (trip_count > 1) and 
# Excluding the last stop on the tour (trip_count != trip_num)
df = pd.merge(
    base_trip[(base_trip.trip_count > 1) & (base_trip.trip_count != base_trip.trip_num)],
    build_trip[(build_trip.trip_count > 1) & (build_trip.trip_count != build_trip.trip_num)],
    how='outer',
    left_index=True,
    right_index=True,
    suffixes=('_base', '_build')
)

In [9]:
df.loc[df.purpose_base == df.purpose_build,'Case'] = "Unchanged"
df.loc[df.purpose_base != df.purpose_build,'Case'] = "Changed"

df.loc[df.purpose_base.isna(),'Case'] = "Newly Created"
df.loc[df.purpose_build.isna(),"Case"] = "Removed"

In [10]:
df_case = df.Case.value_counts()
df_case

Case
Unchanged    7
Name: count, dtype: int64

In [11]:
fig = vh.create_pie_chart(df_case.to_frame().sort_index(),["count"])
fig.show()

## Trip Destination

In [12]:
df = base_trip[base_trip.trip_count != base_trip.trip_num].merge(build_trip[build_trip.trip_count != build_trip.trip_num],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))


In [13]:
df.loc[df.destination_base == df.destination_build,'Case'] = "Unchanged"
df.loc[df.destination_base != df.destination_build,'Case'] = "Changed"

df.loc[df.destination_base.isna(),'Case'] = "Newly Created"
df.loc[df.destination_build.isna(),"Case"] = "Removed"


In [14]:
df = df.Case.value_counts()

df

Case
Unchanged    7
Name: count, dtype: int64

In [15]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Trip Scheduling

In [16]:
print("Lorem ipsum") # Placeholder for WSP

Lorem ipsum


## Trip Mode Choice

In [17]:
print("Lorem ipsum") # Placeholder for CS
# xtab base-vs build mode choice for all trips
# xtab base-vs build mode choice for all trips by tour purpose

Lorem ipsum


In [18]:
df = base_trip[['trip_mode', 'purpose']].merge(build_trip[['trip_mode', 'purpose']],
                                                                 how='outer',
                                                                 left_index=True,
                                                                 right_index=True,
                                                                 suffixes=('_base','_build'))

df.loc[df.trip_mode_base == df.trip_mode_build,'Case'] = "Unchanged"
df.loc[df.trip_mode_base != df.trip_mode_build,'Case'] = "Changed"

df.loc[df.trip_mode_base.isna(),'Case'] = "Newly Created"
df.loc[df.trip_mode_build.isna(),"Case"] = "Removed"

In [19]:
df_case = df.Case.value_counts()
df_case

Case
Unchanged    49
Name: count, dtype: int64

In [20]:
fig = vh.create_pie_chart(df_case.to_frame(), ["count"])
fig.show()

In [21]:
xtab = df[['trip_mode_base', 'trip_mode_build']].value_counts().unstack().fillna(0)
xtab

trip_mode_build,BIKE,DRIVEALONE,SHARED3,TNC_SINGLE,WALK
trip_mode_base,,,,,
BIKE,1.0,0.0,0.0,0.0,0.0
DRIVEALONE,0.0,16.0,0.0,0.0,0.0
SHARED3,0.0,0.0,2.0,0.0,0.0
TNC_SINGLE,0.0,0.0,0.0,5.0,0.0
WALK,0.0,0.0,0.0,0.0,25.0


In [22]:
# | output: asis
# above comment is needed for Quarto to render subtabs correctly

# for now, loop through the the recorded trip purposes
# note that keying against purpose_base will only include trips that exist in base 
all_purps = list(set(df.purpose_base))
all_purps.sort()

for purp in all_purps:
    df_purp = df.loc[df.purpose_base == purp]
    if len(df_purp) == 0:
        continue 
        
    print(f"#### {purp}")
    df_purp_cases = df_purp.Case.value_counts()
    print(str(df_purp_cases).replace("\n","<br>"))
    
    fig = vh.create_pie_chart(df_purp_cases.to_frame().sort_index(),["count"])
    
    fig.show()
    display(Markdown(" "))

#### atwork
Case<br>Unchanged    1<br>Name: count, dtype: int64


#### eatout
Case<br>Unchanged    2<br>Name: count, dtype: int64


#### escort
Case<br>Unchanged    8<br>Name: count, dtype: int64


#### home
Case<br>Unchanged    20<br>Name: count, dtype: int64


#### othdiscr
Case<br>Unchanged    4<br>Name: count, dtype: int64


...

#### school
Case<br>Unchanged    1<br>Name: count, dtype: int64


#### shopping
Case<br>Unchanged    3<br>Name: count, dtype: int64


#### social
Case<br>Unchanged    1<br>Name: count, dtype: int64


#### univ
Case<br>Unchanged    1<br>Name: count, dtype: int64


#### work
Case<br>Unchanged    5<br>Name: count, dtype: int64
